In [2]:
#pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.8 MB/s eta 0:00:00


In [11]:
import pickle
import pandas as pd
import numpy as np
import json

#Text processing and cleaning
import contractions # To include english contractions
import re #regex
import string #used to include punctuation during text processing
from collections import Counter #count strings in texts

#Natural Language Tool Kit NLK package
import nltk
from nltk.corpus import stopwords #Stopwords

import spacy
from spacy.matcher import Matcher


nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodzaraya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rodzaraya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rodzaraya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

All the cell above must be in the aux_func.py file as auxiliary functions

In [10]:
import subprocess

try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    # If the model is not found, download it
    print("Downloading 'en_core_web_sm' model...")
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the '/Users/rodzaraya/.pyenv/versions/3.9.13/bin/python -m pip install --upgrade pip' command.


In [18]:
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity


In [19]:
#pip install scikit-learn

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
def load_skills_from_json(skills_file):
    skills_list = []
    with open(skills_file, 'r') as f:
        data = json.load(f)  # Load the entire JSON file
        for skill in data.keys():  # The keys at the top level are the skill names
            skills_list.append(skill)  # Add the skill to the list
    return skills_list


#=======================================Processing class
class TextPreprocessor:
    def __init__(self, processing_mode='none', custom_punctuation=None, custom_stopwords=None, sentence_analysis=False):
        """
            Initialization considers Custom punctuation, Stop words, and Lemmatizer or Stemmer.
            Updates custom punctuation and custom stop words set with additional ones if provided.
            The processing mode to standardise variants can be choose between none, Stem and Lemma. Each mode is stored in a different
            column of the dataframe.
            Sentence analysis parameter is used to keep the punctuation symbols required for sentence analysis.

            Parameters:
            - processing_mode: String to decide whether to use 'lemma', 'stem', or 'none' for text processing.
            - custom_punctuation: Additional punctuation characters to remove from text.
            - custom_stopwords: Additional stopwords to remove from text.
            - sentence_analysis: Boolean indicating sentence analysis cleaning steps. This mode will keep the punctuation symbols.

            """

        self.punctuation = string.punctuation #Init with all punctuation characters

        if custom_punctuation:
            self.punctuation += custom_punctuation #add custom punctuation

        self.stop_words = set(stopwords.words('english'))
        if custom_stopwords:
            self.stop_words.update(custom_stopwords) #add custom stopwords

        # Determine which text processing mode to use
        self.processing_mode = processing_mode.lower()

        # Set the sentence analysis mode
        self.sentence_analysis = sentence_analysis

        #Set the variant standardization mode
        if self.processing_mode == 'lemma':
            self.lemmatizer = WordNetLemmatizer()
        elif self.processing_mode == 'stem':
            self.stemmer = PorterStemmer()

    # Expand contractions using the contractions library
    def expand_contractions(self, text):
        return contractions.fix(text)

    # Split hyphenated words into separate words, like phone numbers or radio fm, age, etc.
    def split_hyphenated_words(self, text):
        return re.sub(r'-', ' ', text)

    def remove_punctuation(self, text):
        return ''.join([char for char in text if char not in self.punctuation])

    def add_space_after_parenthesis(self, text):
        return re.sub(r'\)', ') ', text)

    def to_lowercase(self, text):
        return text.lower()

    def remove_stopwords(self, text):
        words = word_tokenize(text)
        return ' '.join([word for word in words if word not in self.stop_words])

    def remove_extra_whitespace(self, text):
        return re.sub(r'\s+', ' ', text).strip()

    def stem_words(self, text):
        words = word_tokenize(text)
        return ' '.join([self.stemmer.stem(word) for word in words])

    def lemmatize_words(self, text):
        words = word_tokenize(text)
        return ' '.join([self.lemmatizer.lemmatize(word) for word in words])

    # Order matters
    def preprocess(self, text):
        text = self.expand_contractions(text)
        text = self.split_hyphenated_words(text)
        text = self.add_space_after_parenthesis(text)

        #In case we need to analyse sentences, we will need the punctuations
        if not self.sentence_analysis:
            text = self.remove_punctuation(text)
        text = self.to_lowercase(text)
        #The stopwords are removed if the users wants to standardise variants.
        #If none is selected, the ouput will just perform previous cleaning steps
        if self.processing_mode != 'none':
            text = self.remove_stopwords(text)

        text = self.remove_extra_whitespace(text)

        #Select the processing mode for variants
        if self.processing_mode == 'lemma':
            text = self.lemmatize_words(text)
        elif self.processing_mode == 'stem':
            text = self.stem_words(text)

        return text

    #Apply preprocessing steps to daframe and create a column base on the processing mode
    def preprocess_dataframe(self, df, column_name):
        if not self.sentence_analysis:
            if self.processing_mode == 'lemma':
                df[f'{column_name}_processed_lemma'] = df[column_name].apply(self.preprocess)
            elif self.processing_mode == 'stem':
                df[f'{column_name}_processed_stem'] = df[column_name].apply(self.preprocess)
            else:  # If 'none', apply preprocessing without lemma or stem
                df[f'{column_name}_processed_cleaned'] = df[column_name].apply(self.preprocess)
        else: # Add different processed columns for sentences
            if self.processing_mode == 'lemma':
                df[f'{column_name}_processed_lemma_sent'] = df[column_name].apply(self.preprocess)
            elif self.processing_mode == 'stem':
                df[f'{column_name}_processed_stem_sent'] = df[column_name].apply(self.preprocess)
            else:  # If 'none', apply preprocessing without lemma or stem
                df[f'{column_name}_processed_cleaned_sent'] = df[column_name].apply(self.preprocess)
        return df

#=============================== GET THE EMBEDDINGS

def get_embeddings(text, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().to("cpu").numpy()
    return embeddings


#============================== GET THE SKILLS

# Define a function to apply the matcher and find skills in the text
def find_skills(text):
    doc = nlp(text)
    matches = matcher(doc)
    skills = set()  # To store found skills
    for match_id, start, end in matches:
        skill = doc[start:end].text
        skills.add(skill)
    return skills


def find_top_skills(job_desc_embedding, skill_embeddings, skills_list, threshold=0.55):
    # Ensure that job_desc_embedding is 2D before passing to cosine_similarity
    #job_desc_embedding = np.expand_dims(job_desc_embedding, axis=0)  # Make it 2D
    similarities = cosine_similarity(job_desc_embedding, skill_embeddings).flatten()

    # Find all skills with similarity scores above the threshold
    above_threshold_indices = [i for i, score in enumerate(similarities) if score >= threshold]

    # Get the skills and scores for those above the threshold
    top_skills = [skills_list[i] for i in above_threshold_indices]
    top_scores = [similarities[i] for i in above_threshold_indices]

    # Return both the skills and their similarity scores
    return list(zip(top_skills, top_scores))

#@st.cache_resource
def load_model(model_name):


    # Select device (MPS for Mac, CUDA for NVIDIA GPUs, CPU as a fallback)
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    model = AutoModel.from_pretrained(model_name)
    model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer, device




# LOAD FILES

In [21]:
#Load the files

with open('../data/job_desc_embeddings_skills.pkl', 'rb') as f:
#with open('/content/drive/MyDrive/AT2/data/job_desc_embeddings_skills.pkl', 'rb') as f: #using in colab
    skill_embeddings = pickle.load(f)
    skill_embeddings = np.squeeze(skill_embeddings, axis=1)

with open('../data/job_desc_embeddings.pkl', 'rb') as f:
#with open('/content/drive/MyDrive/AT2/data/job_desc_embeddings.pkl', 'rb') as f:
    job_desc_embeddings = pickle.load(f)


# Load patterns from the JSONL file
skills_patterns = []
with open('../data/jz_skill_patterns.jsonl', 'r') as f:
#with open('/content/drive/MyDrive/AT2/data/jz_skill_patterns.jsonl', 'r') as f:
    for line in f:
        skills_patterns.append(json.loads(line))

skills_list = load_skills_from_json('../data/skills.json')
#skills_list = load_skills_from_json('/content/drive/MyDrive/AT2/data/skills.json')

In [22]:

matcher = Matcher(nlp.vocab)

# Add patterns to the matcher
for pattern in skills_patterns:
    matcher.add(pattern['label'], [pattern['pattern']])


# PREDICT SKILLS

In [23]:
# Example of a new job description
new_job_desc = "We are looking for a data scientist with experience in machine learning and Python and SQL. Knowledge in database and version control and webscrapping."

In [24]:
text_preprocessor = TextPreprocessor(processing_mode='none')

In [25]:
# Preprocess the job description text
new_job_cleaned = text_preprocessor.preprocess(new_job_desc)

In [26]:
new_job_cleaned

'we are looking for a data scientist with experience in machine learning and python and sql knowledge in database and version control and webscrapping'

In [29]:
skills_matched = list(find_skills(new_job_cleaned))
skills_matched

['database', 'machine learning', 'python']

In [35]:
# Load the model and tokenizer only once
model_name = "bert-base-uncased"
# Load the model and tokenizer only once
model, tokenizer, device = load_model(model_name)
job_desc_embedding = get_embeddings(new_job_cleaned, model_name)


In [36]:
THRESHOLD = 0.65
top_skills = find_top_skills(job_desc_embedding, skill_embeddings, skills_list, THRESHOLD)
top_skills

[('agile-project-management', 0.6624161),
 ('amazon-elasticsearch-service', 0.6521224),
 ('amazon-machine-learning', 0.6719476),
 ('amazon-rds-for-postgresql', 0.69189787),
 ('azure-machine-learning', 0.66870147),
 ('beta-by-crashlytics', 0.6519042),
 ('big-data-tools', 0.6576272),
 ('boosting-machine-learning', 0.6513903),
 ('computer-multitasking', 0.6706059),
 ('document-layout-analysis', 0.6584076),
 ('documentation-as-a-service--tools', 0.6722325),
 ('exploratory-data-analysis', 0.66050017),
 ('frameworks-full-stack', 0.6566205),
 ('google-cloud-dataflow', 0.6509979),
 ('humancomputer-information-retrieval', 0.7199203),
 ('humancomputer-interaction', 0.67685163),
 ('java-build-tools', 0.6551467),
 ('js-build-tools--js-task-runners', 0.6532289),
 ('load-and-performance-testing', 0.65299636),
 ('mobile-prototyping--interaction-design-tools', 0.6913935),
 ('mobile-testing-frameworks', 0.66996217),
 ('node.js-process-manager', 0.65105206),
 ('nosql-database-as-a-service', 0.6969453),


In [37]:
skills_list, scores_list = zip(*top_skills)
#format the skills without the -
skills_list = [skill.replace('-', ' ') for skill in skills_list]
skills_list

['agile project management',
 'amazon elasticsearch service',
 'amazon machine learning',
 'amazon rds for postgresql',
 'azure machine learning',
 'beta by crashlytics',
 'big data tools',
 'boosting machine learning',
 'computer multitasking',
 'document layout analysis',
 'documentation as a service  tools',
 'exploratory data analysis',
 'frameworks full stack',
 'google cloud dataflow',
 'humancomputer information retrieval',
 'humancomputer interaction',
 'java build tools',
 'js build tools  js task runners',
 'load and performance testing',
 'mobile prototyping  interaction design tools',
 'mobile testing frameworks',
 'node.js process manager',
 'nosql database as a service',
 'object document mapper odm',
 'object relational mapper orm',
 'open postgresql monitoring',
 'postgresql as a service',
 'self hosted blogging  cms',
 'server configuration and automation',
 'spreadsheets as a backend',
 'sqlalchemy',
 'testing frameworks',
 'tools for text editors',
 'web mining']

In [50]:
total_list = list(set(skills_list + skills_matched))
total_list

['load and performance testing',
 'spreadsheets as a backend',
 'frameworks full stack',
 'nosql database as a service',
 'computer multitasking',
 'document layout analysis',
 'node.js process manager',
 'tools for text editors',
 'amazon rds for postgresql',
 'big data tools',
 'mobile prototyping  interaction design tools',
 'humancomputer information retrieval',
 'js build tools  js task runners',
 'database',
 'azure machine learning',
 'amazon machine learning',
 'self hosted blogging  cms',
 'server configuration and automation',
 'machine learning',
 'boosting machine learning',
 'agile project management',
 'exploratory data analysis',
 'amazon elasticsearch service',
 'mobile testing frameworks',
 'object relational mapper orm',
 'testing frameworks',
 'humancomputer interaction',
 'web mining',
 'postgresql as a service',
 'beta by crashlytics',
 'documentation as a service  tools',
 'java build tools',
 'open postgresql monitoring',
 'google cloud dataflow',
 'object docume

In [39]:
#pip install streamlit